In [3]:
from rdkit import Chem
import pandas as pd

C:\Users\Chaewon\AppData\Local\Temp\ipykernel_8888\853024002.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [4]:
df = pd.read_csv('GPT_response_smiles_dict_fixed_0213.csv')
Lowe_smiles = df['Lowe_smiles'].tolist()
GPT_smiles = df['GPT_smiles'].tolist()
titles = df['nameResolved'].tolist()
paragraphs = df['paragraph'].tolist()
responses = df['json_responses'].tolist()
df.head(3)

,Unnamed: 0,processed_random_samples_Index,temp_compare_Index,documentId,paragraphNum,file_path,headingText,nameResolved,template,productSmiles,paragraph,Lowe_smiles,GPT response,json_responses,smiles_dict,GPT_smiles
0,0,25,49825,US20060252807A1,601.0,C:/2001_Sep2016_USPTOapplications_cml/applicat...,Step 1 4-(4-Ethoxycarbonyl-oxazol-2-ylethynyl)...,4-(4-Ethoxycarbonyl-oxazol-2-ylethynyl)-piperi...,[C:1]#[C:2].[X:6]-[c:3](:[a:4]):[a:5]>>[C:1]#[...,CCOC(=O)c1coc(C#CC2CCN(C(=O)OC(C)(C)C)CC2)n1,A 50 mL round bottom flask was charged with 2-...,[CH2:1]([O:3][C:4]([C:6]1[N:7]=[C:8](Cl)[O:9][...,"{\n ""Detailed Reaction Description"": {\n ""...","{'Detailed Reaction Description': {'Reactants,...","{'A': 'CCOC(=O)C1=COC(=N1)Cl', 'B': 'CC(C)(C)O...",CCOC(=O)C1=COC(=N1)Cl.CC(C)(C)OC(=O)N1CCC(CC1)...
1,1,37,45550,US20050059710A1,108.0,C:/2001_Sep2016_USPTOapplications_cml/applicat...,Step A: Preparation of 2-fluoro-4-[(trimethyls...,2-fluoro-4-[(trimethylsilyl)ethynyl]aniline,[C:1]#[C:2].[X:6]-[c:3](:[a:4]):[a:5]>>[C:1]#[...,C[Si](C)(C)C#Cc1ccc(N)c(F)c1,"2-Fluoro-4-iodoaniline (5.00 g, 21.1 mmol), Cu...",[F:1][C:2]1[CH:8]=[C:7](I)[CH:6]=[CH:5][C:3]=1...,"{\n ""Detailed Reaction Description"": {\n ""...","{'Detailed Reaction Description': {'Reactants,...","{'A': 'C1=CC(=C(C=C1I)F)N', 'B': '[Cu]I', 'C':...",C1=CC(=C(C=C1I)F)N.[Cu]I.C1=CC=C(C=C1)P(C2=CC=...
2,2,25,49827,US20060252807A1,626.0,C:/2001_Sep2016_USPTOapplications_cml/applicat...,4-(4-Ethoxycarbonyl-oxazol-2-ylethynyl)-piperi...,4-(4-Ethoxycarbonyl-oxazol-2-ylethynyl)-piperi...,[C:1]#[C:2].[X:6]-[c:3](:[a:4]):[a:5]>>[C:1]#[...,CCOC(=O)c1coc(C#CC2CCN(C(=O)OC(C)(C)C)CC2)n1,A 50 mL round bottom flask was charged with 2-...,[CH2:1]([O:3][C:4]([C:6]1[N:7]=[C:8](Cl)[O:9][...,"{\n ""Detailed Reaction Description"": {\n ""...","{'Detailed Reaction Description': {'Reactants,...","{'A': 'CCOC(=O)C1=COC(=N1)Cl', 'B': 'CC(C)(C)O...",CCOC(=O)C1=COC(=N1)Cl.CC(C)(C)OC(=O)N1CCC(CC1)...


In [18]:
def demap(smiles, return_role=False):
    mol = Chem.MolFromSmiles(smiles)
    is_reactant = False
    if mol is None:  # Check if the molecule is valid
        return (smiles, False) if return_role else smiles  # Return the original SMILES if invalid

    for atom in mol.GetAtoms():
        if atom.GetAtomMapNum() != 0:
            is_reactant = True #reactant이다
            atom.SetAtomMapNum(0)
    if return_role: #return_role = True인 경우
        # print('is_reactant', is_reactant)
        return Chem.MolToSmiles(mol), is_reactant
    else: # return_role = False인 경우
        return Chem.MolToSmiles(mol)
        
def check_NoSmi(string1, string2, string3):
    check_NoSmi = False
    all_string = string1+string2+string3
    if 'NoSmi' in all_string:
        check_NoSmi = True
    return check_NoSmi

def split_role(rxn):
    reactants, reagents, product = rxn.split('>')
    if not check_NoSmi(reactants,reagents,product):
        if not reagents:
            real_reactants, real_reagents = [], []
        else:
            real_reactants, real_reagents = [], [demap(reagents)]
        for reactant in reactants.split('.'):
            smiles, is_reactant = demap(reactant, True)
            if is_reactant:
                real_reactants.append(smiles)
            else:
                real_reagents.append(smiles)
        return real_reactants, real_reagents, demap(product)
    else:
        print('NoSmi exists in rxn')
        return None
    


In [19]:
def compare_results(i):
    title = titles[i]
    paragraph = paragraphs[i]
    response = responses[i]
    rxn1 = Lowe_smiles[i]
    rxn2 = GPT_smiles[i]
    
    if split_role(rxn2) != None:
        reactants1, reagents1, products1 = split_role(rxn1)
        reactants2, reagents2, products2 = split_role(rxn2)
        rxn1 = '.'.join(sorted(reactants1+reagents1)) + '>>' + products1
        rxn2 = '.'.join(sorted(reactants2+reagents2)) + '>>' + products2
        result = {
            'title': title, 
            'paragraph': paragraph, 
            'response': response,
            'Lowe_rxn': rxn1,
            'GPT_rxn': rxn2,
            'reactant_match': set(reactants1) == set(reactants2),
            'reagent_match': set(reagents1) == set(reagents2), 
            'product_match': products1 == products2,
            'rxn_match': rxn1 == rxn2
        }
        return result
    else:
        print('Error: split_role_NoSmi')
        return None

i = 367
results = compare_results(i)
results

NoSmi exists in rxn
Error: split_role_NoSmi


In [20]:
from tqdm import tqdm

match_cnt = {'product':[], 'reactant':[], 'reagent':[], 'rxn':[]}
no_rxns = []
for i in tqdm(range(len(df)), total=len(df)):
    result = compare_results(i)
    # if 'NoSmi' in result['GPT_rxn']:
    #     no_rxns.append(i)
        # continue
    if result == None:
        no_rxns.append(i)
        continue

    for k in match_cnt.keys():
        if not result['%s_match' % k]:
            match_cnt[k].append(i)
        
#     if not result['GPT_rxn']:
#         match_cnt['no_rxn'].append(i)
        


  0%|          | 0/389 [00:00<?, ?it/s][11:46:21] WARNING: not removing hydrogen atom without neighbors
[11:46:21] WARNING: not removing hydrogen atom without neighbors
[11:46:21] WARNING: not removing hydrogen atom without neighbors
[11:46:21] WARNING: not removing hydrogen atom without neighbors
[11:46:21] WARNING: not removing hydrogen atom without neighbors
[11:46:21] WARNING: not removing hydrogen atom without neighbors
[11:46:21] WARNING: not removing hydrogen atom without neighbors
[11:46:21] WARNING: not removing hydrogen atom without neighbors
[11:46:21] WARNING: not removing hydrogen atom without neighbors
[11:46:21] WARNING: not removing hydrogen atom without neighbors
[11:46:21] WARNING: not removing hydrogen atom without neighbors
[11:46:21] WARNING: not removing hydrogen atom without neighbors
[11:46:21] WARNING: not removing hydrogen atom without neighbors
[11:46:21] WARNING: not removing hydrogen atom without neighbors
[11:46:21] SMILES Parse Error: syntax error while p

NoSmi exists in rxn
Error: split_role_NoSmi
NoSmi exists in rxn
Error: split_role_NoSmi
NoSmi exists in rxn
Error: split_role_NoSmi
NoSmi exists in rxn
Error: split_role_NoSmi
NoSmi exists in rxn
Error: split_role_NoSmi
NoSmi exists in rxn
Error: split_role_NoSmi
NoSmi exists in rxn
Error: split_role_NoSmi
NoSmi exists in rxn
Error: split_role_NoSmi
NoSmi exists in rxn
Error: split_role_NoSmi
NoSmi exists in rxn
Error: split_role_NoSmi
NoSmi exists in rxn
Error: split_role_NoSmi


[11:46:21] WARNING: not removing hydrogen atom without neighbors
[11:46:21] SMILES Parse Error: syntax error while parsing: EinF
[11:46:21] SMILES Parse Error: Failed parsing SMILES 'EinF' for input: 'EinF'
[11:46:21] SMILES Parse Error: syntax error while parsing: EinF
[11:46:21] SMILES Parse Error: Failed parsing SMILES 'EinF' for input: 'EinF'
[11:46:21] SMILES Parse Error: syntax error while parsing: CinD
[11:46:21] SMILES Parse Error: Failed parsing SMILES 'CinD' for input: 'CinD'
[11:46:21] SMILES Parse Error: syntax error while parsing: CinD
[11:46:21] SMILES Parse Error: Failed parsing SMILES 'CinD' for input: 'CinD'
 62%|██████▏   | 241/389 [00:00<00:00, 742.58it/s][11:46:21] WARNING: not removing hydrogen atom without neighbors
[11:46:21] WARNING: not removing hydrogen atom without neighbors
[11:46:21] WARNING: not removing hydrogen atom without neighbors
[11:46:21] WARNING: not removing hydrogen atom without neighbors
[11:46:21] WARNING: not removing hydrogen atom without ne

NoSmi exists in rxn
Error: split_role_NoSmi
NoSmi exists in rxn
Error: split_role_NoSmi
NoSmi exists in rxn
Error: split_role_NoSmi
NoSmi exists in rxn
Error: split_role_NoSmi
NoSmi exists in rxn
Error: split_role_NoSmi
NoSmi exists in rxn
Error: split_role_NoSmi
NoSmi exists in rxn
Error: split_role_NoSmi
NoSmi exists in rxn
Error: split_role_NoSmi
NoSmi exists in rxn
Error: split_role_NoSmi
NoSmi exists in rxn
Error: split_role_NoSmi
NoSmi exists in rxn
Error: split_role_NoSmi
NoSmi exists in rxn
Error: split_role_NoSmi
NoSmi exists in rxn
Error: split_role_NoSmi


[11:46:21] WARNING: not removing hydrogen atom without neighbors
[11:46:21] WARNING: not removing hydrogen atom without neighbors
[11:46:21] WARNING: not removing hydrogen atom without neighbors
[11:46:21] WARNING: not removing hydrogen atom without neighbors
[11:46:21] WARNING: not removing hydrogen atom without neighbors
[11:46:21] WARNING: not removing hydrogen atom without neighbors
[11:46:21] WARNING: not removing hydrogen atom without neighbors
[11:46:21] WARNING: not removing hydrogen atom without neighbors
[11:46:21] WARNING: not removing hydrogen atom without neighbors
[11:46:21] WARNING: not removing hydrogen atom without neighbors
[11:46:21] WARNING: not removing hydrogen atom without neighbors
[11:46:21] WARNING: not removing hydrogen atom without neighbors
[11:46:21] WARNING: not removing hydrogen atom without neighbors
[11:46:21] WARNING: not removing hydrogen atom without neighbors
[11:46:21] WARNING: not removing hydrogen atom without neighbors
[11:46:21] WARNING: not r

{'product': [4, 14, 21, 67, 85, 101, 118, 130, 134, 177, 186, 235, 266, 275, 276, 285, 308, 385], 'reactant': [0, 1, 2, 3, 4, 5, 6, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 92, 94, 95, 96, 97, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 140, 141, 142, 143, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 191, 192, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 20

In [31]:
print(match_cnt)
print('product:', match_cnt['product'])
print('product:', match_cnt['product'])
print(no_rxns)
print(len(df))
print('Cannot get smiles:', len(no_rxns)/len(df)*100, '%')

{'product': [4, 14, 21, 67, 85, 101, 118, 130, 134, 177, 186, 235, 266, 275, 276, 285, 308, 385], 'reactant': [0, 1, 2, 3, 4, 5, 6, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 92, 94, 95, 96, 97, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 140, 141, 142, 143, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 191, 192, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 20

In [35]:
print('Cannot get smiles(no_rxns):', '(%.2f)' % (len(no_rxns)/len(df)))
print ()
for k, v in match_cnt.items():
    vl = len(v)
    print (k, vl, '(%.2f)' % (vl/len(df)), v[:20])

SyntaxError: invalid syntax (1622306863.py, line 1)